In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
# 读取 CSV 数据
df = pd.read_csv("uart_ready_output.csv")
df.columns = ['time', 'value']

In [ ]:
# ✅ 自定义 FIR 低通滤波器权重（简化为 box filter）
def custom_fir_filter(signal, fir_window):
    # Hamming Window Kernel
    # === FPGA FIR 系数 ===
    fpga_kernel = np.array([
        1, 2, 3, 5, 8, 12, 17, 22, 26, 29,
        29, 26, 22, 17, 12, 8, 5, 3, 2, 1
    ])
    fpga_kernel = fpga_kernel / np.sum(fpga_kernel)  # 归一化处理

    filtered = np.convolve(signal, fpga_kernel, mode='same')  # 保持 same 长度
    return filtered

# ✅ 主函数：带独立 MA 和 FIR 窗口大小
def apply_filters(ma_window, fir_window):
    time = df['time'].values
    signal = df['value'].values
    time = np.arange(len(signal))  # fallback index

    # 移动平均
    ma_filtered = pd.Series(signal).rolling(window=ma_window, min_periods=1, center=True).mean()

    # 自定义 FIR 卷积
    fir_filtered = custom_fir_filter(signal, fir_window)

    # 📊 创建 2 个 subplot
    fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
    
    axs[0].plot(time, signal, label='Original', color='gray', linewidth=1)
    axs[0].plot(time, ma_filtered, label=f'MA (window={ma_window})', color='orange', linewidth=1)
    axs[0].set_ylabel("MA Filter")
    axs[0].legend()
    axs[0].grid(True)

    axs[1].plot(time, signal, label='Original', color='gray', linewidth=1)

    axs[1].plot(time, fir_filtered, label=f'FIR (window={fir_window})', color='green', linewidth=1)
    axs[1].set_ylabel("FIR Filter")
    axs[1].set_xlabel("Time")
    axs[1].legend()
    axs[1].grid(True)

    fig.suptitle("MA vs FIR (Custom) Filter Comparison", fontsize=14)
    plt.tight_layout()
    plt.show()

# ✅ 两个独立滑条
ma_slider = widgets.IntSlider(value=5, min=1, max=501, step=2, description='MA Window')
fir_slider = widgets.IntSlider(value=7, min=1, max=501, step=2, description='FIR Window')

# 控件 + 显示
ui = widgets.HBox([ma_slider, fir_slider])
out = widgets.interactive_output(apply_filters, {'ma_window': ma_slider, 'fir_window': fir_slider})
display(ui, out)

Output()